# Network Science: Final Project

## Initialization

This includes:   
* Importing libraries
* Defining functions
* Loading data

### libraries

In [1]:
import networkx as nx #network analysis
import numpy as np 
import matplotlib.pyplot as plt #viz
import random 
import pandas as pd #to manipulate data frames
import math #for logarithm in entropy formula
from IPython.display import display
import os #files manipulation

#if you're working on google colab, uncomment the following line

# from google.colab import drive
# drive.mount('/content/drive')

# import os
# os.chdir('/content/drive/My Drive/net-sci-project-master/net-sci-project-master/code')
# os.getwcd()

### functions

Functions to:  
* compute LCC
* compute I index
* compute R measure
* compute Interval E
* Apply Random Attack
* Apply Targeted Attack (takes a graph and sorted list of nodes and returns dictions of metrics: LCC, I, efficiency)
* Apply Community ased attack

In [2]:
def compute_lcc(G):
    if G.number_of_nodes() == 0:
        return 0
    else:
        max_comp= max(nx.connected_components(G), key=len)
        return len(max_comp)
    
def compute_i(G, i):
    if G.number_of_nodes() == 0:
        return 0
    else:
        max_comp= max(nx.connected_components(G), key=len)
        return len(max_comp)/i

def compute_r(g, sorted_nodes):
    G=g.copy()
    SUM=0
    N= G.number_of_nodes()

    if type(sorted_nodes[0])==tuple:

        for i in range(N):

            n=sorted_nodes[i][0]
            G.remove_node(n)
            lcc= compute_lcc(G)/N
            SUM+=lcc
    else:
        for i in range(N):

            n=sorted_nodes[i]
            G.remove_node(n)
            lcc= compute_lcc(G)/N
            SUM+=lcc

    return SUM/N

def compute_r_50_batches(attack):
    '''takes a tuple of dictionaries (output of attack) that contains the lcc of attack'''
    lcc_attack=attack[3]
    r = sum(list(lcc_attack.values()))/len(list(lcc_attack.values()))
    return r

def compute_r_100_batches(g, sorted_nodes):
    G=g.copy()
    N=G.number_of_nodes()
    batch_size = N//100
    SUM=0
    if type(sorted_nodes[0])==tuple:

        for i in range(50):
            for j in range(batch_size):
                n = sorted_nodes[i*batch_size+j][0]
                G.remove_node(n)
            lcc = compute_lcc(G)/N
            SUM+=lcc
    
    else:
        for i in range(50):
            for j in range(batch_size):
                n = sorted_nodes[i*batch_size+j]
                G.remove_node(n)
            lcc = compute_lcc(G)/N
            SUM+=lcc

    return SUM/100


def random_attack(g):
    G=g.copy()
    nodes = list(G.nodes())
    random.shuffle(nodes)
    #divide the nodes into 100 batches
    fix=G.number_of_nodes()
    batch_size = fix//100
    LCC={}
    Inter={}
    E={}
    lcc_i = max([len(c) for c in nx.connected_components(G)])

    for i in range(50):
        for j in range(batch_size):
            n = nodes[i*batch_size+j]
            G.remove_node(n)
        lcc = compute_lcc(G)
        i_index= compute_i(G, lcc_i)
        LCC[i] = lcc/fix
        Inter[i]=i_index
        E[i]=nx.algorithms.global_efficiency(G)
    return LCC, Inter, E

def random_attack_edge(g):
    '''returns the graph after a random attack of the edge'''
    G = g.copy()
    edges = list(G.edges())
    random.shuffle(edges)
    fix=G.number_of_nodes()
    batch_size = fix//100
    LCC={}
    Inter={}
    E={}
    lcc_i = max([len(c) for c in nx.connected_components(G)])

    for i in range(50):
        for j in range(batch_size):
            e = edges[i*batch_size+j]
            G.remove_edge(e[0], e[1])
        lcc = compute_lcc(G)
        i_index= compute_i(G, lcc_i)
        LCC[i] = lcc/fix
        Inter[i]=i_index
        E[i]=nx.algorithms.global_efficiency(G)
    return LCC, Inter, E

def attack(g, sorted_nodes):
    G=g.copy()
    fix=G.number_of_nodes()
    batch_size = fix//100
    norm_LCC={}
    LCC={}
    Inter={}
    E={}
    lcc_i = max([len(c) for c in nx.connected_components(G)])

    for i in range(50):
        for j in range(batch_size):
            n = sorted_nodes[i*batch_size+j]
            G.remove_node(n)
        lcc = compute_lcc(G)
        i_index= compute_i(G, lcc_i)
        norm_LCC[i] = lcc/fix
        LCC[i]=lcc
        Inter[i]=i_index
        E[i]=nx.algorithms.global_efficiency(G)
    return norm_LCC, Inter, E, LCC


### Loading networks

* Loading the network
* Extract some properties:
  * Compute statistics
  * Get degree distribution

In [6]:
arenas_email = nx.read_gml('../benchmark/arenas-email.gml', label='id')
bn_cat_mixed_species_brain_1 = nx.read_gml('../benchmark/bn-cat-mixed-species_brain_1.gml', label='id')
bn_macaque_rhesus_brain_2 = nx.read_gml('../benchmark/bn-macaque-rhesus_brain_2.gml', label='id')
circuits_s208 = nx.read_gml('../benchmark/circuits s208.gml', label='id')
circuits_s420 = nx.read_gml('../benchmark/circuits s420.gml', label='id')
circuits_s838 = nx.read_gml('../benchmark/circuits s838.gml', label='id')
dolphins = nx.read_gml('../benchmark/dolphins.gml', label='id')
e_coli = nx.read_gml('../benchmark/E. coli.gml', label='id')
facebook_0 = nx.read_gml('../benchmark/facebook 0.gml', label='id')
facebook_107 = nx.read_gml('../benchmark/facebook 107.gml', label='id')
facebook_1684 = nx.read_gml('../benchmark/facebook 1684.gml', label='id')
facebook_348 = nx.read_gml('../benchmark/facebook 348.gml', label='id')
facebook_414 = nx.read_gml('../benchmark/facebook 414.gml', label='id')
facebook_686 = nx.read_gml('../benchmark/facebook 686.gml', label='id')
fb_pages_food = nx.read_gml('../benchmark/fb-pages-food.gml', label='id')
karate = nx.read_gml('../benchmark/Karate.gml', label='id')
polbooks = nx.read_gml('../benchmark/polbooks.gml', label='id')
soc_firm_hi_tech = nx.read_gml('../benchmark/soc-firm-hi-tech.gml', label='id')
soc_tribes = nx.read_gml('../benchmark/soc-tribes.gml', label='id')
word_adjacencies = nx.read_gml('../benchmark/word_adjacencies.gml', label='id')

net_911=nx.read_gml('../benchmark/911.gml', label='id')
corruption=nx.read_gml('../benchmark/corruption.gml', label='id')
crime_net=nx.read_gml('../benchmark/CrimeNet.gml', label='id')
digg=nx.read_gml('../benchmark/Digg.gml', label='id')
email=nx.read_gml('../benchmark/Email.gml', label='id')
jazz=nx.read_gml('../benchmark/Jazz.gml', label='id')
petster_hamster=nx.read_gml('../benchmark/Petster-Hamster.gml', label='id')
router=nx.read_gml('../benchmark/Router.gml', label='id')

## Attacks

### Random Attack

Random attacks applied to all the networks, all are divided to 100 batches, attack is up to 50% of the nodes/edges:
* nodes attacks 
* edges attacks

In [ ]:
#--------------------------------------------------------------------------------------------------------------#
#nodes random attacks:

random_arenas_email_attack = random_attack(arenas_email)
random_bn_cat_mixed_species_brain_1_attack = random_attack(bn_cat_mixed_species_brain_1)
random_bn_macaque_rhesus_brain_2_attack = random_attack(bn_macaque_rhesus_brain_2)
random_circuits_s208_attack = random_attack(circuits_s208)
random_circuits_s420_attack = random_attack(circuits_s420)
random_circuits_s838_attack = random_attack(circuits_s838)
random_dolphins_attack = random_attack(dolphins)
random_e_coli_attack = random_attack(e_coli)
random_facebook_0_attack = random_attack(facebook_0)
random_facebook_107_attack = random_attack(facebook_107)
random_facebook_1684_attack = random_attack(facebook_1684)
random_facebook_348_attack = random_attack(facebook_348)
random_facebook_414_attack = random_attack(facebook_414)
random_facebook_686_attack = random_attack(facebook_686)
random_fb_pages_food_attack = random_attack(fb_pages_food)
random_karate_attack = random_attack(karate)
random_polbooks_attack = random_attack(polbooks)
random_soc_firm_hi_tech_attack = random_attack(soc_firm_hi_tech)
random_soc_tribes_attack = random_attack(soc_tribes)
random_word_adjacencies_attack = random_attack(word_adjacencies)

random_net911_attack = random_attack(net_911)
random_corruption_attack = random_attack(corruption)
random_crime_net_attack = random_attack(crime_net)
random_email_attack = random_attack(email)
random_jazz_attack = random_attack(jazz)
random_petster_hamster_attack = random_attack(petster_hamster)
random_router_attack = random_attack(router)
# random_digg_attack = random_attack(digg)

#--------------------------------------------------------------------------------------------------------------#
#edges random attacks:

random_arenas_email_attack_edge= random_attack_edge(arenas_email)
random_bn_cat_mixed_species_brain_1_attack_edge= random_attack_edge(bn_cat_mixed_species_brain_1)
random_bn_macaque_rhesus_brain_2_attack_edge= random_attack_edge(bn_macaque_rhesus_brain_2)
random_circuits_s208_attack_edge= random_attack_edge(circuits_s208)
random_circuits_s420_attack_edge= random_attack_edge(circuits_s420)
random_circuits_s838_attack_edge= random_attack_edge(circuits_s838)
random_dolphins_attack_edge= random_attack_edge(dolphins)
random_e_coli_attack_edge= random_attack_edge(e_coli)
random_facebook_0_attack_edge= random_attack_edge(facebook_0)
random_facebook_107_attack_edge= random_attack_edge(facebook_107)
random_facebook_1684_attack_edge= random_attack_edge(facebook_1684)
random_facebook_348_attack_edge= random_attack_edge(facebook_348)
random_facebook_414_attack_edge= random_attack_edge(facebook_414)
random_facebook_686_attack_edge= random_attack_edge(facebook_686)
random_fb_pages_food_attack_edge= random_attack_edge(fb_pages_food)
random_karate_attack_edge= random_attack_edge(karate)
random_polbooks_attack_edge= random_attack_edge(polbooks)
random_soc_firm_hi_tech_attack_edge= random_attack_edge(soc_firm_hi_tech)
random_soc_tribes_attack_edge= random_attack_edge(soc_tribes)
random_word_adjacencies_attack_edge= random_attack_edge(word_adjacencies)

random_net911_attack_edge= random_attack_edge(net_911)
random_corruption_attack_edge= random_attack_edge(corruption)
random_crime_net_attack_edge= random_attack_edge(crime_net)
random_email_attack_edge= random_attack_edge(email)
random_jazz_attack_edge= random_attack_edge(jazz)
random_petster_hamster_attack_edge= random_attack_edge(petster_hamster)
random_router_attack_edge= random_attack_edge(router)
# random_digg_attack_edge= random_attack_edge(digg)

#--------------------------------------------------------------------------------------------------------------#

### Centrality-based Targeted Attack
---------------------------

* Nodes centralities:
  * Betweenness centrality
  * Closeness centrality
  * Degree centrality
  * PLCi centrality
  * Percolaion centrality
  * Proximity centrality
  * Mapping Entropy Betweenness centrality
  * Mapping Entropy Closeness centrality
  * Mapping Entropy Degree centrality   

----------------

* Edges centralities:
  * Betweenness centrality


-----------------

#### Node centralities:

In [ ]:
betweenness_centrality_sorted_arenas_email = sorted(nx.betweenness_centrality(arenas_email).items(), key=lambda x: x[1], reverse=True)
betweenness_centrality_sorted_bn_cat_mixed_species_brain_1 = sorted(nx.betweenness_centrality(bn_cat_mixed_species_brain_1).items(), key=lambda x: x[1], reverse=True)
betweenness_centrality_sorted_bn_macaque_rhesus_brain_2 = sorted(nx.betweenness_centrality(bn_macaque_rhesus_brain_2).items(), key=lambda x: x[1], reverse=True)
betweenness_centrality_sorted_circuits_s208 = sorted(nx.betweenness_centrality(circuits_s208).items(), key=lambda x: x[1], reverse=True)
betweenness_centrality_sorted_circuits_s420 = sorted(nx.betweenness_centrality(circuits_s420).items(), key=lambda x: x[1], reverse=True)
betweenness_centrality_sorted_circuits_s838 = sorted(nx.betweenness_centrality(circuits_s838).items(), key=lambda x: x[1], reverse=True) 
betweenness_centrality_sorted_dolphins = sorted(nx.betweenness_centrality(dolphins).items(), key=lambda x: x[1], reverse=True)
betweenness_centrality_sorted_e_coli = sorted(nx.betweenness_centrality(e_coli).items(), key=lambda x: x[1], reverse=True)
betweenness_centrality_sorted_facebook_0 = sorted(nx.betweenness_centrality(facebook_0).items(), key=lambda x: x[1], reverse=True)
betweenness_centrality_sorted_facebook_107 = sorted(nx.betweenness_centrality(facebook_107).items(), key=lambda x: x[1], reverse=True)
betweenness_centrality_sorted_facebook_1684 = sorted(nx.betweenness_centrality(facebook_1684).items(), key=lambda x: x[1], reverse=True)
betweenness_centrality_sorted_facebook_348 = sorted(nx.betweenness_centrality(facebook_348).items(), key=lambda x: x[1], reverse=True)
betweenness_centrality_sorted_facebook_414 = sorted(nx.betweenness_centrality(facebook_414).items(), key=lambda x: x[1], reverse=True)
betweenness_centrality_sorted_facebook_686 = sorted(nx.betweenness_centrality(facebook_686).items(), key=lambda x: x[1], reverse=True)
betweenness_centrality_sorted_fb_pages_food = sorted(nx.betweenness_centrality(fb_pages_food).items(), key=lambda x: x[1], reverse=True)
betweenness_centrality_sorted_karate = sorted(nx.betweenness_centrality(karate).items(), key=lambda x: x[1], reverse=True)   
betweenness_centrality_sorted_polbooks = sorted(nx.betweenness_centrality(polbooks).items(), key=lambda x: x[1], reverse=True)
betweenness_centrality_sorted_soc_firm_hi_tech = sorted(nx.betweenness_centrality(soc_firm_hi_tech).items(), key=lambda x: x[1], reverse=True)
betweenness_centrality_sorted_soc_tribes = sorted(nx.betweenness_centrality(soc_tribes).items(), key=lambda x: x[1], reverse=True)
betweenness_centrality_sorted_word_adjacencies = sorted(nx.betweenness_centrality(word_adjacencies).items(), key=lambda x: x[1], reverse=True)
betweenness_centrality_sorted_net911 = sorted(nx.betweenness_centrality(net_911).items(), key=lambda x: x[1], reverse=True)
betweenness_centrality_sorted_corruption = sorted(nx.betweenness_centrality(corruption).items(), key=lambda x: x[1], reverse=True)
betweenness_centrality_sorted_crime_net = sorted(nx.betweenness_centrality(crime_net).items(), key=lambda x: x[1], reverse=True)
# betweenness_centrality_sorted_digg = sorted(nx.betweenness_centrality(digg).items(), key=lambda x: x[1], reverse=True)
betweenness_centrality_jazz = sorted(nx.betweenness_centrality(jazz).items(), key=lambda x: x[1], reverse=True)
betweenness_centrality_sorted_petster_hamster = sorted(nx.betweenness_centrality(petster_hamster).items(), key=lambda x: x[1], reverse=True)
betweenness_centrality_sorted_router = sorted(nx.betweenness_centrality(router).items(), key=lambda x: x[1], reverse=True)
betweenness_centrality_sorted_email = sorted(nx.betweenness_centrality(email).items(), key=lambda x: x[1], reverse=True)


closeness_centrality_sorted_arenas_email = sorted(nx.closeness_centrality(arenas_email).items(), key=lambda x: x[1], reverse=True)
closeness_centrality_sorted_bn_cat_mixed_species_brain_1 = sorted(nx.closeness_centrality(bn_cat_mixed_species_brain_1).items(), key=lambda x: x[1], reverse=True)
closeness_centrality_sorted_bn_macaque_rhesus_brain_2 = sorted(nx.closeness_centrality(bn_macaque_rhesus_brain_2).items(), key=lambda x: x[1], reverse=True)
closeness_centrality_sorted_circuits_s208 = sorted(nx.closeness_centrality(circuits_s208).items(), key=lambda x: x[1], reverse=True)
closeness_centrality_sorted_circuits_s420 = sorted(nx.closeness_centrality(circuits_s420).items(), key=lambda x: x[1], reverse=True)
closeness_centrality_sorted_circuits_s838 = sorted(nx.closeness_centrality(circuits_s838).items(), key=lambda x: x[1], reverse=True) 
closeness_centrality_sorted_dolphins = sorted(nx.closeness_centrality(dolphins).items(), key=lambda x: x[1], reverse=True)
closeness_centrality_sorted_e_coli = sorted(nx.closeness_centrality(e_coli).items(), key=lambda x: x[1], reverse=True)
closeness_centrality_sorted_facebook_0 = sorted(nx.closeness_centrality(facebook_0).items(), key=lambda x: x[1], reverse=True)
closeness_centrality_sorted_facebook_107 = sorted(nx.closeness_centrality(facebook_107).items(), key=lambda x: x[1], reverse=True)
closeness_centrality_sorted_facebook_1684 = sorted(nx.closeness_centrality(facebook_1684).items(), key=lambda x: x[1], reverse=True)
closeness_centrality_sorted_facebook_348 = sorted(nx.closeness_centrality(facebook_348).items(), key=lambda x: x[1], reverse=True)
closeness_centrality_sorted_facebook_414 = sorted(nx.closeness_centrality(facebook_414).items(), key=lambda x: x[1], reverse=True)
closeness_centrality_sorted_facebook_686 = sorted(nx.closeness_centrality(facebook_686).items(), key=lambda x: x[1], reverse=True)
closeness_centrality_sorted_fb_pages_food = sorted(nx.closeness_centrality(fb_pages_food).items(), key=lambda x: x[1], reverse=True)
closeness_centrality_sorted_karate = sorted(nx.closeness_centrality(karate).items(), key=lambda x: x[1], reverse=True)   
closeness_centrality_sorted_polbooks = sorted(nx.closeness_centrality(polbooks).items(), key=lambda x: x[1], reverse=True)
closeness_centrality_sorted_soc_firm_hi_tech = sorted(nx.closeness_centrality(soc_firm_hi_tech).items(), key=lambda x: x[1], reverse=True)
closeness_centrality_sorted_soc_tribes = sorted(nx.closeness_centrality(soc_tribes).items(), key=lambda x: x[1], reverse=True)
closeness_centrality_sorted_word_adjacencies = sorted(nx.closeness_centrality(word_adjacencies).items(), key=lambda x: x[1], reverse=True)
closeness_centrality_sorted_net911 = sorted(nx.closeness_centrality(net_911).items(), key=lambda x: x[1], reverse=True)
closeness_centrality_sorted_corruption = sorted(nx.closeness_centrality(corruption).items(), key=lambda x: x[1], reverse=True)
closeness_centrality_sorted_crime_net = sorted(nx.closeness_centrality(crime_net).items(), key=lambda x: x[1], reverse=True)
# closeness_centrality_sorted_digg = sorted(nx.closeness_centrality(digg).items(), key=lambda x: x[1], reverse=True)
closeness_centrality_jazz = sorted(nx.closeness_centrality(jazz).items(), key=lambda x: x[1], reverse=True)
closeness_centrality_sorted_petster_hamster = sorted(nx.closeness_centrality(petster_hamster).items(), key=lambda x: x[1], reverse=True)
closeness_centrality_sorted_router = sorted(nx.closeness_centrality(router).items(), key=lambda x: x[1], reverse=True)
closeness_centrality_sorted_email = sorted(nx.closeness_centrality(email).items(), key=lambda x: x[1], reverse=True)


degree_centrality_sorted_arenas_email = sorted(nx.degree_centrality(arenas_email).items(), key=lambda x: x[1], reverse=True)
degree_centrality_sorted_bn_cat_mixed_species_brain_1 = sorted(nx.degree_centrality(bn_cat_mixed_species_brain_1).items(), key=lambda x: x[1], reverse=True)
degree_centrality_sorted_bn_macaque_rhesus_brain_2 = sorted(nx.degree_centrality(bn_macaque_rhesus_brain_2).items(), key=lambda x: x[1], reverse=True) 
degree_centrality_sorted_circuits_s208 = sorted(nx.degree_centrality(circuits_s208).items(), key=lambda x: x[1], reverse=True)
degree_centrality_sorted_circuits_s420 = sorted(nx.degree_centrality(circuits_s420).items(), key=lambda x: x[1], reverse=True)
degree_centrality_sorted_circuits_s838 = sorted(nx.degree_centrality(circuits_s838).items(), key=lambda x: x[1], reverse=True)
degree_centrality_sorted_dolphins = sorted(nx.degree_centrality(dolphins).items(), key=lambda x: x[1], reverse=True)
degree_centrality_sorted_e_coli = sorted(nx.degree_centrality(e_coli).items(), key=lambda x: x[1], reverse=True)
degree_centrality_sorted_facebook_0 = sorted(nx.degree_centrality(facebook_0).items(), key=lambda x: x[1], reverse=True)
degree_centrality_sorted_facebook_107 = sorted(nx.degree_centrality(facebook_107).items(), key=lambda x: x[1], reverse=True)
degree_centrality_sorted_facebook_1684 = sorted(nx.degree_centrality(facebook_1684).items(), key=lambda x: x[1], reverse=True)
degree_centrality_sorted_facebook_348 = sorted(nx.degree_centrality(facebook_348).items(), key=lambda x: x[1], reverse=True)
degree_centrality_sorted_facebook_414 = sorted(nx.degree_centrality(facebook_414).items(), key=lambda x: x[1], reverse=True)
degree_centrality_sorted_facebook_686 = sorted(nx.degree_centrality(facebook_686).items(), key=lambda x: x[1], reverse=True)
degree_centrality_sorted_fb_pages_food = sorted(nx.degree_centrality(fb_pages_food).items(), key=lambda x: x[1], reverse=True)
degree_centrality_sorted_karate = sorted(nx.degree_centrality(karate).items(), key=lambda x: x[1], reverse=True)
degree_centrality_sorted_polbooks = sorted(nx.degree_centrality(polbooks).items(), key=lambda x: x[1], reverse=True)
degree_centrality_sorted_soc_firm_hi_tech = sorted(nx.degree_centrality(soc_firm_hi_tech).items(), key=lambda x: x[1], reverse=True)
degree_centrality_sorted_soc_tribes = sorted(nx.degree_centrality(soc_tribes).items(), key=lambda x: x[1], reverse=True)
degree_centrality_sorted_word_adjacencies = sorted(nx.degree_centrality(word_adjacencies).items(), key=lambda x: x[1], reverse=True)
degree_centrality_sorted_net911 = sorted(nx.degree_centrality(net_911).items(), key=lambda x: x[1], reverse=True)
degree_centrality_sorted_corruption = sorted(nx.degree_centrality(corruption).items(), key=lambda x: x[1], reverse=True)
degree_centrality_sorted_crime_net = sorted(nx.degree_centrality(crime_net).items(), key=lambda x: x[1], reverse=True)
# degree_centrality_sorted_digg = sorted(nx.degree_centrality(digg).items(), key=lambda x: x[1], reverse=True)
degree_centrality_jazz = sorted(nx.degree_centrality(jazz).items(), key=lambda x: x[1], reverse=True)
degree_centrality_sorted_petster_hamster = sorted(nx.degree_centrality(petster_hamster).items(), key=lambda x: x[1], reverse=True)
degree_centrality_sorted_router = sorted(nx.degree_centrality(router).items(), key=lambda x: x[1], reverse=True)
degree_centrality_sorted_email = sorted(nx.degree_centrality(email).items(), key=lambda x: x[1], reverse=True)


bet_attack_arenas_email = attack(arenas_email, [x[0] for x in betweenness_centrality_sorted_arenas_email])
bet_attack_bn_cat_mixed_species_brain_1 = attack(bn_cat_mixed_species_brain_1, [x[0] for x in betweenness_centrality_sorted_bn_cat_mixed_species_brain_1])
bet_attack_bn_macaque_rhesus_brain_2 = attack(bn_macaque_rhesus_brain_2, [x[0] for x in betweenness_centrality_sorted_bn_macaque_rhesus_brain_2])
bet_attack_circuits_s208 = attack(circuits_s208, [x[0] for x in betweenness_centrality_sorted_circuits_s208])
bet_attack_circuits_s420 = attack(circuits_s420, [x[0] for x in betweenness_centrality_sorted_circuits_s420])
bet_attack_circuits_s838 = attack(circuits_s838, [x[0] for x in betweenness_centrality_sorted_circuits_s838])
bet_attack_dolphins = attack(dolphins, [x[0] for x in betweenness_centrality_sorted_dolphins])
bet_attack_e_coli = attack(e_coli, [x[0] for x in betweenness_centrality_sorted_e_coli])
bet_attack_facebook_0 = attack(facebook_0, [x[0] for x in betweenness_centrality_sorted_facebook_0])
bet_attack_facebook_107 = attack(facebook_107, [x[0] for x in betweenness_centrality_sorted_facebook_107])
bet_attack_facebook_1684 = attack(facebook_1684, [x[0] for x in betweenness_centrality_sorted_facebook_1684])
bet_attack_facebook_348 = attack(facebook_348, [x[0] for x in betweenness_centrality_sorted_facebook_348])
bet_attack_facebook_414 = attack(facebook_414, [x[0] for x in betweenness_centrality_sorted_facebook_414])
bet_attack_facebook_686 = attack(facebook_686, [x[0] for x in betweenness_centrality_sorted_facebook_686])
bet_attack_fb_pages_food = attack(fb_pages_food, [x[0] for x in betweenness_centrality_sorted_fb_pages_food])
bet_attack_karate = attack(karate, [x[0] for x in betweenness_centrality_sorted_karate])
bet_attack_polbooks = attack(polbooks, [x[0] for x in betweenness_centrality_sorted_polbooks])
bet_attack_soc_firm_hi_tech = attack(soc_firm_hi_tech, [x[0] for x in betweenness_centrality_sorted_soc_firm_hi_tech])
bet_attack_soc_tribes = attack(soc_tribes, [x[0] for x in betweenness_centrality_sorted_soc_tribes])
bet_attack_word_adjacencies = attack(word_adjacencies, [x[0] for x in betweenness_centrality_sorted_word_adjacencies])
bet_attack_net911 = attack(net_911, [x[0] for x in betweenness_centrality_sorted_net911])
bet_attack_corruption = attack(corruption, [x[0] for x in betweenness_centrality_sorted_corruption])
bet_attack_crime_net = attack(crime_net, [x[0] for x in betweenness_centrality_sorted_crime_net])
# bet_attack_digg = attack(digg, [x[0] for x in betweenness_centrality_sorted_digg])
bet_attack_jazz = attack(jazz, [x[0] for x in betweenness_centrality_jazz])
bet_attack_petster_hamster = attack(petster_hamster, [x[0] for x in betweenness_centrality_sorted_petster_hamster])
bet_attack_router = attack(router, [x[0] for x in betweenness_centrality_sorted_router])
bet_attack_email = attack(email, [x[0] for x in betweenness_centrality_sorted_email])

clos_attack_arenas_email = attack(arenas_email, [x[0] for x in closeness_centrality_sorted_arenas_email])
clos_attack_bn_cat_mixed_species_brain_1 = attack(bn_cat_mixed_species_brain_1, [x[0] for x in closeness_centrality_sorted_bn_cat_mixed_species_brain_1])
clos_attack_bn_macaque_rhesus_brain_2 = attack(bn_macaque_rhesus_brain_2, [x[0] for x in closeness_centrality_sorted_bn_macaque_rhesus_brain_2])
clos_attack_circuits_s208 = attack(circuits_s208, [x[0] for x in closeness_centrality_sorted_circuits_s208])
clos_attack_circuits_s420 = attack(circuits_s420, [x[0] for x in closeness_centrality_sorted_circuits_s420])
clos_attack_circuits_s838 = attack(circuits_s838, [x[0] for x in closeness_centrality_sorted_circuits_s838])
clos_attack_dolphins = attack(dolphins, [x[0] for x in closeness_centrality_sorted_dolphins])
clos_attack_e_coli = attack(e_coli, [x[0] for x in closeness_centrality_sorted_e_coli])
clos_attack_facebook_0 = attack(facebook_0, [x[0] for x in closeness_centrality_sorted_facebook_0])
clos_attack_facebook_107 = attack(facebook_107, [x[0] for x in closeness_centrality_sorted_facebook_107])
clos_attack_facebook_1684 = attack(facebook_1684, [x[0] for x in closeness_centrality_sorted_facebook_1684])
clos_attack_facebook_348 = attack(facebook_348, [x[0] for x in closeness_centrality_sorted_facebook_348])
clos_attack_facebook_414 = attack(facebook_414, [x[0] for x in closeness_centrality_sorted_facebook_414])
clos_attack_facebook_686 = attack(facebook_686, [x[0] for x in closeness_centrality_sorted_facebook_686])
clos_attack_fb_pages_food = attack(fb_pages_food, [x[0] for x in closeness_centrality_sorted_fb_pages_food])
clos_attack_karate = attack(karate, [x[0] for x in closeness_centrality_sorted_karate])
clos_attack_polbooks = attack(polbooks, [x[0] for x in closeness_centrality_sorted_polbooks])
clos_attack_soc_firm_hi_tech = attack(soc_firm_hi_tech, [x[0] for x in closeness_centrality_sorted_soc_firm_hi_tech])
clos_attack_soc_tribes = attack(soc_tribes, [x[0] for x in closeness_centrality_sorted_soc_tribes])
clos_attack_word_adjacencies = attack(word_adjacencies, [x[0] for x in closeness_centrality_sorted_word_adjacencies])
clos_attack_net911 = attack(net_911, [x[0] for x in closeness_centrality_sorted_net911])
clos_attack_corruption = attack(corruption, [x[0] for x in closeness_centrality_sorted_corruption])
clos_attack_crime_net = attack(crime_net, [x[0] for x in closeness_centrality_sorted_crime_net])
# clos_attack_digg = attack(digg, [x[0] for x in closeness_centrality_sorted_digg])
clos_attack_jazz = attack(jazz, [x[0] for x in closeness_centrality_jazz])
clos_attack_petster_hamster = attack(petster_hamster, [x[0] for x in closeness_centrality_sorted_petster_hamster])
clos_attack_router = attack(router, [x[0] for x in closeness_centrality_sorted_router])
clos_attack_email = attack(email, [x[0] for x in closeness_centrality_sorted_email])

deg_attack_arenas_email = attack(arenas_email, [x[0] for x in degree_centrality_sorted_arenas_email])
deg_attack_bn_cat_mixed_species_brain_1 = attack(bn_cat_mixed_species_brain_1, [x[0] for x in degree_centrality_sorted_bn_cat_mixed_species_brain_1])
deg_attack_bn_macaque_rhesus_brain_2 = attack(bn_macaque_rhesus_brain_2, [x[0] for x in degree_centrality_sorted_bn_macaque_rhesus_brain_2])
deg_attack_circuits_s208 = attack(circuits_s208, [x[0] for x in degree_centrality_sorted_circuits_s208])
deg_attack_circuits_s420 = attack(circuits_s420, [x[0] for x in degree_centrality_sorted_circuits_s420])
deg_attack_circuits_s838 = attack(circuits_s838, [x[0] for x in degree_centrality_sorted_circuits_s838])
deg_attack_dolphins = attack(dolphins, [x[0] for x in degree_centrality_sorted_dolphins])
deg_attack_e_coli = attack(e_coli, [x[0] for x in degree_centrality_sorted_e_coli])
deg_attack_facebook_0 = attack(facebook_0, [x[0] for x in degree_centrality_sorted_facebook_0])
deg_attack_facebook_107 = attack(facebook_107, [x[0] for x in degree_centrality_sorted_facebook_107])
deg_attack_facebook_1684 = attack(facebook_1684, [x[0] for x in degree_centrality_sorted_facebook_1684])
deg_attack_facebook_348 = attack(facebook_348, [x[0] for x in degree_centrality_sorted_facebook_348])
deg_attack_facebook_414 = attack(facebook_414, [x[0] for x in degree_centrality_sorted_facebook_414])
deg_attack_facebook_686 = attack(facebook_686, [x[0] for x in degree_centrality_sorted_facebook_686])
deg_attack_fb_pages_food = attack(fb_pages_food, [x[0] for x in degree_centrality_sorted_fb_pages_food])
deg_attack_karate = attack(karate, [x[0] for x in degree_centrality_sorted_karate])
deg_attack_polbooks = attack(polbooks, [x[0] for x in degree_centrality_sorted_polbooks])
deg_attack_soc_firm_hi_tech = attack(soc_firm_hi_tech, [x[0] for x in degree_centrality_sorted_soc_firm_hi_tech])
deg_attack_soc_tribes = attack(soc_tribes, [x[0] for x in degree_centrality_sorted_soc_tribes])
deg_attack_word_adjacencies = attack(word_adjacencies, [x[0] for x in degree_centrality_sorted_word_adjacencies])
deg_attack_net911 = attack(net_911, [x[0] for x in degree_centrality_sorted_net911])
deg_attack_corruption = attack(corruption, [x[0] for x in degree_centrality_sorted_corruption])
deg_attack_crime_net = attack(crime_net, [x[0] for x in degree_centrality_sorted_crime_net])
# deg_attack_digg = attack(digg, [x[0] for x in degree_centrality_sorted_digg])
deg_attack_jazz = attack(jazz, [x[0] for x in degree_centrality_jazz])
deg_attack_petster_hamster = attack(petster_hamster, [x[0] for x in degree_centrality_sorted_petster_hamster])
deg_attack_router = attack(router, [x[0] for x in degree_centrality_sorted_router])
deg_attack_email = attack(email, [x[0] for x in degree_centrality_sorted_email])



In [ ]:
arenas_email_complex_df = pd.read_csv('../results/resultsarenas_email.csv', skiprows=1, header=None)
plci_arenas_email = dict(zip(arenas_email_complex_df[0], arenas_email_complex_df[1]))
plci_arenas_email_sorted = sorted(plci_arenas_email.items(), key=lambda x: x[1], reverse=True)
plci_arenas_email_nodes =[x[0]-1 for x in plci_arenas_email_sorted]
percolation_arenas_email = dict(zip(arenas_email_complex_df[0], arenas_email_complex_df[2]))
percolation_arenas_email_sorted = sorted(percolation_arenas_email.items(), key=lambda x: x[1], reverse=True)
percolation_arenas_email_nodes =[x[0]-1 for x in percolation_arenas_email_sorted]

bn_cat_mixed_species_brain_1_complex_df = pd.read_csv('../results/resultsbn_cat_mixed_species_brain_1.csv', skiprows=1, header=None)
plci_bn_cat_mixed_species_brain_1 = dict(zip(bn_cat_mixed_species_brain_1_complex_df[0], bn_cat_mixed_species_brain_1_complex_df[1]))
plci_bn_cat_mixed_species_brain_1_sorted = sorted(plci_bn_cat_mixed_species_brain_1.items(), key=lambda x: x[1], reverse=True)
plci_bn_cat_mixed_species_brain_1_nodes =[x[0]-1 for x in plci_bn_cat_mixed_species_brain_1_sorted]
percolation_bn_cat_mixed_species_brain_1 = dict(zip(bn_cat_mixed_species_brain_1_complex_df[0], bn_cat_mixed_species_brain_1_complex_df[2]))
percolation_bn_cat_mixed_species_brain_1_sorted = sorted(percolation_bn_cat_mixed_species_brain_1.items(), key=lambda x: x[1], reverse=True)
percolation_bn_cat_mixed_species_brain_1_nodes =[x[0]-1 for x in percolation_bn_cat_mixed_species_brain_1_sorted]

bn_macaque_rhesus_brain_2_complex_df = pd.read_csv('../results/resultsbn_macaque_rhesus_brain_2.csv', skiprows=1, header=None)
plci_bn_macaque_rhesus_brain_2 = dict(zip(bn_macaque_rhesus_brain_2_complex_df[0], bn_macaque_rhesus_brain_2_complex_df[1]))
plci_bn_macaque_rhesus_brain_2_sorted = sorted(plci_bn_macaque_rhesus_brain_2.items(), key=lambda x: x[1], reverse=True)
plci_bn_macaque_rhesus_brain_2_nodes =[x[0]-1 for x in plci_bn_macaque_rhesus_brain_2_sorted]
percolation_bn_macaque_rhesus_brain_2 = dict(zip(bn_macaque_rhesus_brain_2_complex_df[0], bn_macaque_rhesus_brain_2_complex_df[2]))
percolation_bn_macaque_rhesus_brain_2_sorted = sorted(percolation_bn_macaque_rhesus_brain_2.items(), key=lambda x: x[1], reverse=True)
percolation_bn_macaque_rhesus_brain_2_nodes =[x[0]-1 for x in percolation_bn_macaque_rhesus_brain_2_sorted]

circuits_s208_complex_df = pd.read_csv('../results/resultscircuits s208.csv', skiprows=1, header=None)
plci_circuits_s208 = dict(zip(circuits_s208_complex_df[0], circuits_s208_complex_df[1]))
plci_circuits_s208_sorted = sorted(plci_circuits_s208.items(), key=lambda x: x[1], reverse=True)
plci_circuits_s208_nodes =[x[0]-1 for x in plci_circuits_s208_sorted]
percolation_circuits_s208 = dict(zip(circuits_s208_complex_df[0], circuits_s208_complex_df[2]))
percolation_circuits_s208_sorted = sorted(percolation_circuits_s208.items(), key=lambda x: x[1], reverse=True)
percolation_circuits_s208_nodes =[x[0]-1 for x in percolation_circuits_s208_sorted]

circuits_s420_complex_df = pd.read_csv('../results/resultscircuits s420.csv', skiprows=1, header=None)
plci_circuits_s420 = dict(zip(circuits_s420_complex_df[0], circuits_s420_complex_df[1]))
plci_circuits_s420_sorted = sorted(plci_circuits_s420.items(), key=lambda x: x[1], reverse=True)
plci_circuits_s420_nodes =[x[0]-1 for x in plci_circuits_s420_sorted]
percolation_circuits_s420 = dict(zip(circuits_s420_complex_df[0], circuits_s420_complex_df[2]))
percolation_circuits_s420_sorted = sorted(percolation_circuits_s420.items(), key=lambda x: x[1], reverse=True)
percolation_circuits_s420_nodes =[x[0]-1 for x in percolation_circuits_s420_sorted]

circuits_s838_complex_df = pd.read_csv('../results/resultscircuits s838.csv', skiprows=1, header=None)
plci_circuits_s838 = dict(zip(circuits_s838_complex_df[0], circuits_s838_complex_df[1]))
plci_circuits_s838_sorted = sorted(plci_circuits_s838.items(), key=lambda x: x[1], reverse=True)
plci_circuits_s838_nodes =[x[0]-1 for x in plci_circuits_s838_sorted]
percolation_circuits_s838 = dict(zip(circuits_s838_complex_df[0], circuits_s838_complex_df[2]))
percolation_circuits_s838_sorted = sorted(percolation_circuits_s838.items(), key=lambda x: x[1], reverse=True)
percolation_circuits_s838_nodes =[x[0]-1 for x in percolation_circuits_s838_sorted]

dolphins_complex_df = pd.read_csv('../results/resultsdolphins.csv', skiprows=1, header=None)
plci_dolphins = dict(zip(dolphins_complex_df[0], dolphins_complex_df[1]))
plci_dolphins_sorted = sorted(plci_dolphins.items(), key=lambda x: x[1], reverse=True)
plci_dolphins_nodes =[x[0]-1 for x in plci_dolphins_sorted]
percolation_dolphins = dict(zip(dolphins_complex_df[0], dolphins_complex_df[2]))
percolation_dolphins_sorted = sorted(percolation_dolphins.items(), key=lambda x: x[1], reverse=True)
percolation_dolphins_nodes =[x[0]-1 for x in percolation_dolphins_sorted]

e_coli_complex_df = pd.read_csv('../results/resultsE. coli.csv', skiprows=1, header=None)
plci_e_coli = dict(zip(e_coli_complex_df[0], e_coli_complex_df[1]))
plci_e_coli_sorted = sorted(plci_e_coli.items(), key=lambda x: x[1], reverse=True)
plci_e_coli_nodes =[x[0]-1 for x in plci_e_coli_sorted]
percolation_e_coli = dict(zip(e_coli_complex_df[0], e_coli_complex_df[2]))
percolation_e_coli_sorted = sorted(percolation_e_coli.items(), key=lambda x: x[1], reverse=True)
percolation_e_coli_nodes =[x[0]-1 for x in percolation_e_coli_sorted]

facebook_0_complex_df = pd.read_csv('../results/resultsfacebook 0.csv', skiprows=1, header=None)
plci_facebook_0 = dict(zip(facebook_0_complex_df[0], facebook_0_complex_df[1]))
plci_facebook_0_sorted = sorted(plci_facebook_0.items(), key=lambda x: x[1], reverse=True)
plci_facebook_0_nodes =[x[0]-1 for x in plci_facebook_0_sorted]
percolation_facebook_0 = dict(zip(facebook_0_complex_df[0], facebook_0_complex_df[2]))
percolation_facebook_0_sorted = sorted(percolation_facebook_0.items(), key=lambda x: x[1], reverse=True)
percolation_facebook_0_nodes =[x[0]-1 for x in percolation_facebook_0_sorted]

facebook_107_complex_df = pd.read_csv('../results/resultsfacebook 107.csv', skiprows=1, header=None)
plci_facebook_107 = dict(zip(facebook_107_complex_df[0], facebook_107_complex_df[1]))
plci_facebook_107_sorted = sorted(plci_facebook_107.items(), key=lambda x: x[1], reverse=True)
plci_facebook_107_nodes =[x[0]-1 for x in plci_facebook_107_sorted]
percolation_facebook_107 = dict(zip(facebook_107_complex_df[0], facebook_107_complex_df[2]))
percolation_facebook_107_sorted = sorted(percolation_facebook_107.items(), key=lambda x: x[1], reverse=True)
percolation_facebook_107_nodes =[x[0]-1 for x in percolation_facebook_107_sorted]

facebook_1684_complex_df = pd.read_csv('../results/resultsfacebook 1684.csv', skiprows=1, header=None)
plci_facebook_1684 = dict(zip(facebook_1684_complex_df[0], facebook_1684_complex_df[1]))
plci_facebook_1684_sorted = sorted(plci_facebook_1684.items(), key=lambda x: x[1], reverse=True)
plci_facebook_1684_nodes =[x[0]-1 for x in plci_facebook_1684_sorted]
percolation_facebook_1684 = dict(zip(facebook_1684_complex_df[0], facebook_1684_complex_df[2]))
percolation_facebook_1684_sorted = sorted(percolation_facebook_1684.items(), key=lambda x: x[1], reverse=True)
percolation_facebook_1684_nodes =[x[0]-1 for x in percolation_facebook_1684_sorted]

facebook_348_complex_df = pd.read_csv('../results/resultsfacebook 348.csv', skiprows=1, header=None)
plci_facebook_348 = dict(zip(facebook_348_complex_df[0], facebook_348_complex_df[1]))
plci_facebook_348_sorted = sorted(plci_facebook_348.items(), key=lambda x: x[1], reverse=True)
plci_facebook_348_nodes =[x[0]-1 for x in plci_facebook_348_sorted]
percolation_facebook_348 = dict(zip(facebook_348_complex_df[0], facebook_348_complex_df[2]))
percolation_facebook_348_sorted = sorted(percolation_facebook_348.items(), key=lambda x: x[1], reverse=True)
percolation_facebook_348_nodes =[x[0]-1 for x in percolation_facebook_348_sorted]

facebook_414_complex_df = pd.read_csv('../results/resultsfacebook 414.csv', skiprows=1, header=None)
plci_facebook_414 = dict(zip(facebook_414_complex_df[0], facebook_414_complex_df[1]))
plci_facebook_414_sorted = sorted(plci_facebook_414.items(), key=lambda x: x[1], reverse=True)
plci_facebook_414_nodes =[x[0]-1 for x in plci_facebook_414_sorted]
percolation_facebook_414 = dict(zip(facebook_414_complex_df[0], facebook_414_complex_df[2]))
percolation_facebook_414_sorted = sorted(percolation_facebook_414.items(), key=lambda x: x[1], reverse=True)
percolation_facebook_414_nodes =[x[0]-1 for x in percolation_facebook_414_sorted]

facebook_686_complex_df = pd.read_csv('../results/resultsfacebook 686.csv', skiprows=1, header=None)
plci_facebook_686 = dict(zip(facebook_686_complex_df[0], facebook_686_complex_df[1]))
plci_facebook_686_sorted = sorted(plci_facebook_686.items(), key=lambda x: x[1], reverse=True)  
plci_facebook_686_nodes =[x[0]-1 for x in plci_facebook_686_sorted]
percolation_facebook_686 = dict(zip(facebook_686_complex_df[0], facebook_686_complex_df[2]))
percolation_facebook_686_sorted = sorted(percolation_facebook_686.items(), key=lambda x: x[1], reverse=True)    
percolation_facebook_686_nodes =[x[0]-1 for x in percolation_facebook_686_sorted]

fb_pages_food_complex_df = pd.read_csv('../results/resultsfb_pages_food.csv', skiprows=1, header=None)
plci_fb_pages_food = dict(zip(fb_pages_food_complex_df[0], fb_pages_food_complex_df[1]))
plci_fb_pages_food_sorted = sorted(plci_fb_pages_food.items(), key=lambda x: x[1], reverse=True)
plci_fb_pages_food_nodes =[x[0]-1 for x in plci_fb_pages_food_sorted]
percolation_fb_pages_food = dict(zip(fb_pages_food_complex_df[0], fb_pages_food_complex_df[2]))
percolation_fb_pages_food_sorted = sorted(percolation_fb_pages_food.items(), key=lambda x: x[1], reverse=True)
percolation_fb_pages_food_nodes =[x[0]-1 for x in percolation_fb_pages_food_sorted]

karate_complex_df = pd.read_csv('../results/resultskarate.csv', skiprows=1, header=None)
plci_karate = dict(zip(karate_complex_df[0], karate_complex_df[1])) 
plci_karate_sorted = sorted(plci_karate.items(), key=lambda x: x[1], reverse=True)
plci_karate_nodes =[x[0]-1 for x in plci_karate_sorted]
percolation_karate = dict(zip(karate_complex_df[0], karate_complex_df[2]))
percolation_karate_sorted = sorted(percolation_karate.items(), key=lambda x: x[1], reverse=True)
percolation_karate_nodes =[x[0]-1 for x in percolation_karate_sorted]

polbooks_complex_df = pd.read_csv('../results/resultspolbooks.csv', skiprows=1, header=None)
plci_polbooks = dict(zip(polbooks_complex_df[0], polbooks_complex_df[1]))
plci_polbooks_sorted = sorted(plci_polbooks.items(), key=lambda x: x[1], reverse=True)
plci_polbooks_nodes =[x[0]-1 for x in plci_polbooks_sorted]
percolation_polbooks = dict(zip(polbooks_complex_df[0], polbooks_complex_df[2]))
percolation_polbooks_sorted = sorted(percolation_polbooks.items(), key=lambda x: x[1], reverse=True)
percolation_polbooks_nodes =[x[0]-1 for x in percolation_polbooks_sorted]

soc_firm_hi_tech_complex_df = pd.read_csv('../results/resultssoc_firm_hi_tech.csv', skiprows=1, header=None)
plci_soc_firm_hi_tech = dict(zip(soc_firm_hi_tech_complex_df[0], soc_firm_hi_tech_complex_df[1]))
plci_soc_firm_hi_tech_sorted = sorted(plci_soc_firm_hi_tech.items(), key=lambda x: x[1], reverse=True)
plci_soc_firm_hi_tech_nodes =[x[0]-1 for x in plci_soc_firm_hi_tech_sorted]
percolation_soc_firm_hi_tech = dict(zip(soc_firm_hi_tech_complex_df[0], soc_firm_hi_tech_complex_df[2]))
percolation_soc_firm_hi_tech_sorted = sorted(percolation_soc_firm_hi_tech.items(), key=lambda x: x[1], reverse=True)
percolation_soc_firm_hi_tech_nodes =[x[0]-1 for x in percolation_soc_firm_hi_tech_sorted]

soc_tribes_complex_df = pd.read_csv('../results/resultssoc_tribes.csv', skiprows=1, header=None)
plci_soc_tribes = dict(zip(soc_tribes_complex_df[0], soc_tribes_complex_df[1]))
plci_soc_tribes_sorted = sorted(plci_soc_tribes.items(), key=lambda x: x[1], reverse=True)
plci_soc_tribes_nodes =[x[0]-1 for x in plci_soc_tribes_sorted]
percolation_soc_tribes = dict(zip(soc_tribes_complex_df[0], soc_tribes_complex_df[2]))
percolation_soc_tribes_sorted = sorted(percolation_soc_tribes.items(), key=lambda x: x[1], reverse=True)
percolation_soc_tribes_nodes =[x[0]-1 for x in percolation_soc_tribes_sorted]

word_adjacencies_complex_df = pd.read_csv('../results/resultsword_adjacencies.csv', skiprows=1, header=None)
plci_word_adjacencies = dict(zip(word_adjacencies_complex_df[0], word_adjacencies_complex_df[1]))
plci_word_adjacencies_sorted = sorted(plci_word_adjacencies.items(), key=lambda x: x[1], reverse=True)
plci_word_adjacencies_nodes =[x[0]-1 for x in plci_word_adjacencies_sorted]
percolation_word_adjacencies = dict(zip(word_adjacencies_complex_df[0], word_adjacencies_complex_df[2]))
percolation_word_adjacencies_sorted = sorted(percolation_word_adjacencies.items(), key=lambda x: x[1], reverse=True)
percolation_word_adjacencies_nodes =[x[0]-1 for x in percolation_word_adjacencies_sorted]

